# Caching with `gw-signal-tools`

NOTE: please be aware that the chosen scheme only caches arguments given to waveform generator. Default arguments will NOT be cached in this way, so if there is e.g. conditioning involved, this might cause some inconsistency for cached and non-cached generator, even if the same set of parameters is passed!

How it works:

In [1]:
import astropy.units as u

from gw_signal_tools import enable_caching, disable_caching
from gw_signal_tools.types import HashableDict
from gw_signal_tools.waveform import get_wf_generator

enable_caching()  # Uncomment to enable caching
# disable_caching()  # To make sure caching is disabled
# -- If one of the lines is (un-)comented here, kernel has to be restarted

/Users/maxmelching/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS
2025-12-20  22:47:00  INFO (caching.py: 58): Enabling caching


The pyseobnr package has failed to load, you will not be able to employ SEOBNRv5 approximants.


In [2]:
f_min = 20.0 * u.Hz
f_max = 1024.0 * u.Hz

wf_params = {
    'mass1': 50.0 * u.solMass,
    'mass2': 50 * u.solMass,
    'f22_start': f_min,
    'f_max': f_max,
    'f22_ref': f_min,
    'phi_ref': 0.0 * u.rad,
    'distance': 1.0 * u.Mpc,
    'inclination': 0.0 * u.rad,
    'condition': 0,
}

hashable_wf_params = HashableDict(wf_params)

Note: for calls to be hashable, one has to add the cache, but also define waveform dictionaries as a ``HashableDict``. A priori, dictionaries are not hashable and adding it to this fundamental type would not feel like a good coding practice. Thus we subclass it and add properties in the new class.

In [3]:
wf_gen = get_wf_generator('IMRPhenomXPHM')

/Users/maxmelching/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/lalsimulation/gwsignal/core/waveform.py:226: UserWarning: This code is currently UNREVIEWED, use with caution!
  warnings.warn("This code is currently UNREVIEWED, use with caution!")


In [4]:
from functools import cache

wf_gen_with_caching = cache(get_wf_generator('IMRPhenomXPHM'))
# -- NOTE: any cacher can be used herer as wrapper
# -- Equivalent: wf_gen_with_caching = get_wf_generator('IMRPhenomXPHM', cache=True)

The following calls will always work

In [5]:
wf_gen_with_caching(hashable_wf_params)
wf_gen(hashable_wf_params)

<FrequencySeries([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j,
                  0.+0.j]
                 unit=Unit("s strain"),
                 f0=<Quantity 0. Hz>,
                 df=<Quantity 0.0625 Hz>,
                 epoch=<Time object: scale='utc' format='gps' value=-15.999999999989711>,
                 name='hplus',
                 channel=None)>

This call only works if caching has been enabled immediately at the beginning of the file

In [6]:
wf_gen(wf_params)

TypeError: unhashable type: 'dict'

Unfortunately, running disable here has no effect if it has been enabled at the beginning (because the waveform generators have been defined in the "enabled"-state)

In [7]:
disable_caching()

2025-12-20  22:47:05  INFO (caching.py: 46): Disabling caching


In [8]:
wf_gen(wf_params)

TypeError: unhashable type: 'dict'

A new import, on the other hand, does have an effect and causes no error when called with the usual, i.e. non-hashable, dictionary. That is because the function checks the current caching status upon each call separately.

In [9]:
from gw_signal_tools.waveform import get_wf_generator

new_wf_gen = get_wf_generator('IMRPhenomXPHM')

In [10]:
new_wf_gen(wf_params)

<FrequencySeries([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j,
                  0.+0.j]
                 unit=Unit("s strain"),
                 f0=<Quantity 0. Hz>,
                 df=<Quantity 0.0625 Hz>,
                 epoch=<Time object: scale='utc' format='gps' value=-15.999999999989711>,
                 name='hplus',
                 channel=None)>